**The task**: to decrypt given message using MCMC approach. 

**Message**: "щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд \
бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз"

# Preparation

In [80]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Libraries

In [81]:
import string
import math
import random

Sample text for scoring statistics

In [82]:
path_to_language_corpus_fold = '/content/drive/My Drive/Magistracy_SPbU/3_semester/Machine_Learning/MCMC/MCMC_decryption/'

with open(path_to_language_corpus_fold + 'war_and_peace_ru.txt', 'r', encoding = 'utf-8') as f:
    raw_text = f.read().lower()

In [83]:
raw_text[:1000]

'том первый\nчасть первая\ni\n– eh bien, mon prince. gênes et lucques ne sont plus que des apanages, des поместья, de la famille buonaparte. non, je vous préviens que si vous ne me dites pas que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocités de cet antichrist (ma parole, j’y crois) – je ne vous connais plus, vous n’êtes plus mon ami, vous n’êtes plus мой верный раб, comme vous dites.[1] ну, здравствуйте, здравствуйте. je vois que je vous fais peur,[2] садитесь и рассказывайте.\n\nтак говорила в июле 1805 года известная анна павловна шерер, фрейлина и приближенная императрицы марии феодоровны, встречая важного и чиновного князя василия, первого приехавшего на ее вечер. анна павловна кашляла несколько дней, у нее был грипп, как она говорила (грипп был тогда новое слово, употреблявшееся только редкими). в записочках, разосланных утром с красным лакеем, было написано без различия во всех:\n\n«si vous n’avez rien de mieux а faire, mo

Creating alphabet

In [84]:
a_cyrillic = ord('а')
alphabet_list = ''.join([chr(i).upper() for i in range(a_cyrillic,a_cyrillic+6)] + [chr(a_cyrillic+33)] + [chr(i) for i in range(a_cyrillic+6,a_cyrillic+32)]) + ' '
alphabet_list = alphabet_list.upper()

In [85]:
alphabet_list

'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ '

# Functions

In [86]:
# This function takes as input a decryption key and creates a dict for key where each letter in the decryption key
# maps to a alphabet For example if the decryption key is "ПНСЕО...." this function will create a dict like {П:А,Н:Б,С:В....}
def create_cipher_dict(cipher):
    cipher_dict = {}
    for i in range(len(cipher)):
        cipher_dict[alphabet_list[i]] = cipher[i]
    return cipher_dict

# This function takes a text and applies the cipher/key on the text and returns text.
def apply_cipher_on_text(text,cipher):
    cipher_dict = create_cipher_dict(cipher) 
    text = list(text)
    newtext = ""
    for elem in text:
        if elem.upper() in cipher_dict:
            newtext+=cipher_dict[elem.upper()]
        else:
            newtext+=" "
    return newtext

# This function takes as input a path to a long text and creates scoring_params dict which contains the 
# number of time each pair of alphabet appears together
# Ex. {'AБ':234,'КЛ':2343,'ЖЗ':23 ..}
def create_scoring_params_dict(longtext_path):
    scoring_params = {}
    with open(longtext_path) as fp:
        for line in fp:
            data = list(line.strip())
            for i in range(len(data)-1):
                alpha_i = data[i].upper()
                alpha_j = data[i+1].upper()
                # cleaning text 
                if alpha_i not in alphabet_list and alpha_i != " ":
                    alpha_i = " "
                if alpha_j not in alphabet_list and alpha_j != " ":
                    alpha_j = " "
                key = alpha_i+alpha_j
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
    return scoring_params

# This function takes as input a text and creates scoring_params dict which contains the 
# number of time each pair of alphabet appears together
# Ex. {'АБ':234,'КЛ':2343,'ЖЗ':23 ..}
def score_params_on_cipher(text):
    scoring_params = {}
    data = list(text.strip())
    for i in range(len(data)-1):
        alpha_i =data[i].upper()
        alpha_j = data[i+1].upper()
        if alpha_i not in alphabet_list and alpha_i != " ":
            alpha_i = " "
        if alpha_j not in alphabet_list and alpha_j != " ":
            alpha_j = " "
        key = alpha_i+alpha_j
        if key in scoring_params:
            scoring_params[key]+=1
        else:
            scoring_params[key]=1
    return scoring_params

# This function takes the text to be decrypted and a cipher to score the cipher.
# This function returns the log(score) metric
def get_cipher_score(text,cipher,scoring_params):
    cipher_dict = create_cipher_dict(cipher)
    decrypted_text = apply_cipher_on_text(text,cipher)
    scored_f = score_params_on_cipher(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items():
        if k in scoring_params:
            cipher_score += v*math.log(scoring_params[k])
    return cipher_score


Normalization

In [87]:
# This function takes as input a path to a long text and creates scoring_params dict which contains the 
# normalized number of time each pair of alphabet appears together
def create_normalized_scoring_params_dict(longtext_path):
    scoring_params = {}
    with open(longtext_path) as fp:
        # text_len = 
        for line in fp:
            data = list(line.strip())
            for i in range(len(data)-1):
                alpha_i = data[i].upper()
                alpha_j = data[i+1].upper()
                if alpha_i not in alphabet_list and alpha_i != " ":
                    alpha_i = " "
                if alpha_j not in alphabet_list and alpha_j != " ":
                    alpha_j = " "
                key = alpha_i+alpha_j
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
    # normalization
    appearance_num = scoring_params.values()
    total_appearance_num = sum(appearance_num)
    scoring_params = {k: v / total_appearance_num for k, v in scoring_params.items()}
    return scoring_params

# This function takes as input a text and creates scoring_params dict which contains the 
# normalized number of time each pair of alphabet appears together
def normalized_score_params_on_cipher(text):
    scoring_params = {}
    data = list(text.strip())
    for i in range(len(data)-1):
        alpha_i =data[i].upper()
        alpha_j = data[i+1].upper()
        if alpha_i not in alphabet_list and alpha_i != " ":
            alpha_i = " "
        if alpha_j not in alphabet_list and alpha_j != " ":
            alpha_j = " "
        key = alpha_i+alpha_j
        if key in scoring_params:
            scoring_params[key]+=1
        else:
            scoring_params[key]=1
    # normalization
    appearance_num = scoring_params.values()
    total_appearance_num = sum(appearance_num)
    scoring_params = {k: v / total_appearance_num for k, v in scoring_params.items()}
    return scoring_params

def normalized_get_cipher_score(text,cipher,scoring_params):
    cipher_dict = create_cipher_dict(cipher)
    decrypted_text = apply_cipher_on_text(text,cipher)
    # scored_f = score_params_on_cipher(decrypted_text)
    scored_f = normalized_score_params_on_cipher(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items():
        if k in scoring_params:
            cipher_score += v*math.log(scoring_params[k])
            # cipher_score += v*scoring_params[k]
            # cipher_score += v**scoring_params[k]
    return cipher_score

In [88]:
# Generate a proposal cipher by swapping letters at two random location
def generate_cipher(cipher):
    pos1 = random.randint(0, len(list(cipher))-1)
    pos2 = random.randint(0, len(list(cipher))-1)
    if pos1 == pos2:
        return generate_cipher(cipher)
    else:
        cipher = list(cipher)
        pos1_alpha = cipher[pos1]
        pos2_alpha = cipher[pos2]
        cipher[pos1] = pos2_alpha
        cipher[pos2] = pos1_alpha
        return "".join(cipher)

# Toss a random coin with probability of head p. If coin comes head return true else false.
def random_coin(p):
    unif = random.uniform(0,1)
    if unif>=p:
        return False
    else:
        return True

In [89]:
# Takes as input a text to decrypt and runs a MCMC algorithm for n_iter. Returns the state having maximum score and also
# the last few states 
def MCMC_decrypt(n_iter,cipher_text,scoring_params):
    current_cipher = alphabet_list
    state_keeper = set()
    best_state = ''
    score = 0
    for i in range(n_iter):
        state_keeper.add(current_cipher)
        # generate new cipher
        proposed_cipher = generate_cipher(current_cipher)
        # score current and proposed cipher 
        score_current_cipher = get_cipher_score(cipher_text,current_cipher,scoring_params)
        score_proposed_cipher = get_cipher_score(cipher_text,proposed_cipher,scoring_params)
        # compare scores of 2 ciphers
        acceptance_probability = min(1,math.exp(score_proposed_cipher-score_current_cipher))
        if score_current_cipher>score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i%20000==0:
            print("iter",i,":",apply_cipher_on_text(cipher_text,current_cipher)[0:99])
    return state_keeper,best_state

def MCMC_decrypt_norm(n_iter,cipher_text,scoring_params):
    current_cipher = alphabet_list
    state_keeper = set()
    best_state = ''
    score = 0
    for i in range(n_iter):
        state_keeper.add(current_cipher)
        # generate new cipher
        proposed_cipher = generate_cipher(current_cipher)
        # score current and proposed cipher 
        score_current_cipher = normalized_get_cipher_score(cipher_text,current_cipher,scoring_params)
        score_proposed_cipher = normalized_get_cipher_score(cipher_text,proposed_cipher,scoring_params)
        # compare scores of 2 ciphers
        acceptance_probability = min(1,math.exp(score_proposed_cipher-score_current_cipher))
        if score_current_cipher>score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i%20000==0:
            print("iter",i,":",apply_cipher_on_text(cipher_text,current_cipher)[0:99])
    return state_keeper,best_state

# Main part

Decyphering the message using "War and Peace"

In [90]:
## Run the Main Program:
scoring_params = create_scoring_params_dict(path_to_language_corpus_fold + 'war_and_peace_ru.txt')

cipher_text = "щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд \
бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз"

print("Text To Decode:", cipher_text)
print("\n")
states,best_state = MCMC_decrypt(1000000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:",apply_cipher_on_text(cipher_text,best_state))
print("\n")
print("MCMC KEY FOUND:",best_state)

Text To Decode: щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз


iter 0 : ЩДШЭСЗДБГДТИЫЛЪИРДВСДПИРВЯСНДЖДЯГФГДОГЪЗД БГДСЫНЪЬИСЗДЕГЯСЗДРГСДОВЯГ БИЪЬИСЗДНТДБНЁДЩДЫИФВСИАДШСВДС
iter 20000 : Я ЧУТЬ НЕ ЗАРЫЛАД ОТ ШАДОСТИ К СЕБЕ ВЕЛЬ ГНЕ ТРИЛЖАТЬ ФЕСТЬ ДЕТ ВОСЕГНАЛЖАТЬ ИЗ НИХ Я РАБОТАМ ЧТО Т
iter 40000 : Я ЧУТЬ НЕ ЗАРЫЛАД ОТ БАДОСТИ К СЕГЕ ВЕЛЬ МНЕ ТРИЛЖАТЬ ШЕСТЬ ДЕТ ВОСЕМНАЛЖАТЬ ИЗ НИХ Я РАГОТАЮ ЧТО Т
iter 60000 : Я ЧУТЬ НЕ ЗАРЫЛАД ОТ ШАДОСТИ К СЕГЕ ВЕЛЬ МНЕ ТРИЛЖАТЬ ЩЕСТЬ ДЕТ ВОСЕМНАЛЖАТЬ ИЗ НИХ Я РАГОТАЮ ЧТО Т
iter 80000 : Я ЧУТЬ НЕ ЗАРЫЛАД ОТ БАДОСТИ К СЕГЕ ВЕЛЬ МНЕ ТРИЛЖАТЬ ЦЕСТЬ ДЕТ ВОСЕМНАЛЖАТЬ ИЗ НИХ Я РАГОТАЮ ЧТО Т
iter 100000 : Я ЧУТЬ НЕ ЗАРЫЛАД ОТ ЦАДОСТИ К СЕБЕ ВЕЛЬ МНЕ ТРИЛЖАТЬ ФЕСТЬ ДЕТ ВОСЕМНАЛЖАТЬ ИЗ НИХ Я РАБОТАЮ ЧТО Т
iter 120000 : Я ЧУТЬ НЕ ЗАРЫЛА

In [76]:
decyphered_text = apply_cipher_on_text(cipher_text,best_state)
decyphered_text

'Я ЧУТЬ НЕ ЗАРЫЛАД ОТ ЦАДОСТИ К СЕБЕ ВЕЛЬ МНЕ ТРИЛЖАТЬ ЩЕСТЬ ДЕТ ВОСЕМНАЛЖАТЬ ИЗ НИХ Я РАБОТАЮ ЧТО ТО ЗАРАБАТЫВАЮ ПОКУПАЮ ВДАЛЕЮ КАК МНЕ ПРЕЛСТАВДЯДОСЬ НЕКОТОРОЙ СОБСТВЕННОСТЬЮ И В РЕЗУДЬТАТЕ  ОЛИН ЧЕМОЛАН ПРИЧЕМ ЛОВОДЬНО СКРОМНОГО РАЗМЕРА ВЫХОЛИТ Я НИШИЙ КАК ЦЕ ЭТО ПОДУЧИДОСЬ'

Now let's score statistics on the text of Crime and Punishment and compare the results. 

In [74]:
## Run the Main Program:
scoring_params = create_scoring_params_dict(path_to_language_corpus_fold + 'crime_and_punishment_ru.txt')

cipher_text = "щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд \
бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз"

print("Text To Decode:", cipher_text)
print("\n")
states,best_state = MCMC_decrypt(1000000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:",apply_cipher_on_text(cipher_text,best_state))
print("\n")
print("MCMC KEY FOUND:",best_state)

Text To Decode: щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз


iter 0 : ЩДШЭСЗДБГДТИЫЛЪИРДВСДПИРВЯСНДЖДЯГФГДОГЪЗД БГДСЫНЪЬИСЗДЕГЯСЗДРГСДОВЯГ БИЪЬИСЗДНТДБНЁДЩДЫИФВСИАДШСВДС
iter 20000 : Я ЧУТЬ НЕ ЗАРЫДАЛ ОТ ЖАЛОСТИ К СЕБЕ ВЕДЬ МНЕ ТРИДЦАТЬ ШЕСТЬ ЛЕТ ВОСЕМНАДЦАТЬ ИЗ НИХ Я РАБОТАЮ ЧТО Т
iter 40000 : Я ЧУТЬ НЕ ЗАРЫДАЛ ОТ ЖАЛОСТИ К СЕБЕ ВЕДЬ МНЕ ТРИДЦАТЬ ФЕСТЬ ЛЕТ ВОСЕМНАДЦАТЬ ИЗ НИХ Я РАБОТАЮ ЧТО Т
iter 60000 : Я ЧУТЬ НЕ ЗАРЫДАЛ ОТ ЖАЛОСТИ К СЕБЕ ВЕДЬ МНЕ ТРИДЦАТЬ ЩЕСТЬ ЛЕТ ВОСЕМНАДЦАТЬ ИЗ НИХ Я РАБОТАЮ ЧТО Т
iter 80000 : Я ЧУТЬ НЕ ЗАРЫДАЛ ОТ ЖАЛОСТИ К СЕБЕ ВЕДЬ МНЕ ТРИДЦАТЬ ФЕСТЬ ЛЕТ ВОСЕМНАДЦАТЬ ИЗ НИХ Я РАБОТАЮ ЧТО Т
iter 100000 : Я ЧУТЬ НЕ ЗАРЫДАЛ ОТ ЖАЛОСТИ К СЕБЕ ВЕДЬ МНЕ ТРИДЦАТЬ ФЕСТЬ ЛЕТ ВОСЕМНАДЦАТЬ ИЗ НИХ Я РАБОТАЮ ЧТО Т
iter 120000 : Я ЧУТЬ НЕ ЗАРЫДА

# Conclusion

As we can see from results obtained from 2 experiments using differents texts for scoring statistics: there is no particular difference in volume of training text. In the both cases the number of iterations to find the solution is comparable.

As we can see the encrypted message is the part of the Dovlatov novel "The Suitcase (Chemodan)": "Я чуть не зарыдал от жалости к себе. Ведь мне тридцать шесть лет. Восемнадцать из них я работаю. Что-то зарабатываю, покупаю. Владею, как мне представлялось, некоторой собственностью. И в результате – один чемодан. Причем довольно скромного размера. Выходит, я нищий? Как же это получилось?!"


